## Preprocess MAANG companies stock market data

- Import libraries  
- Load raw data (CSV, API, etc.)  
- Handle missing values / duplicates  
- Feature engineering  
- Save processed data to file (e.g., `data/processed.csv`)

Dataset by SOUMENDRA PRASAD MOHANTY on Kaggle: https://www.kaggle.com/datasets/soumendraprasad/stock

In [11]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import os

### Load all csv files for different companies

In [12]:
raw_AAPL = pd.read_csv('data/raw/Apple.csv')
raw_AAPL.head(3)

,Open,High,Low,Close,Adj Close,Volume,Date
0,0.936384,1.004464,0.907924,0.999442,0.850643,535796800,2000-01-03
1,0.966518,0.987723,0.903460,0.915179,0.778926,512377600,2000-01-04
2,0.926339,0.987165,0.919643,0.928571,0.790324,778321600,2000-01-05


In [13]:
# load all CSV files
data_path = "data/raw"
company_names = ["Microsoft", "Apple", "Amazon", "Netflix", "Google"]

dfs = {}

for name in company_names:
    file_path = os.path.join(data_path, f"{name}.csv")
    df = pd.read_csv(file_path)
    df["Company"] = name  # add identifier column
    dfs[name] = df

In [14]:
dfs["Microsoft"].head(3)

,Open,High,Low,Close,Adj Close,Volume,Date,Company
0,58.68750,59.3125,56.0000,58.28125,36.361576,53228400,2000-01-03,Microsoft
1,56.78125,58.5625,56.1250,56.31250,35.133263,54119000,2000-01-04,Microsoft
2,55.56250,58.1875,54.6875,56.90625,35.503712,64059600,2000-01-05,Microsoft


### Combine files into one Dataframe

In [15]:
combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df.sample(3)

,Open,High,Low,Close,Adj Close,Volume,Date,Company
5782,236.110001,238.869995,233.940002,238.729996,238.133545,21207000,2022-12-23,Microsoft
21636,384.380005,386.799988,373.829987,377.140015,377.140015,8638700,2018-10-02,Netflix
22699,289.989990,300.660004,286.500000,297.959991,297.959991,6376200,2022-12-21,Netflix


### Basic Preprocessing

Check for NaN values

Set date column to pandas datetime

Remove unwanted columns

In [16]:
# check for missing values 
combined_df.isna().any().any()

np.False_

In [17]:
# pd date time and sort values
combined_df["Date"] = pd.to_datetime(combined_df["Date"])
combined_df = combined_df.sort_values(["Date"])

In [18]:
# remove close column, we will use adjusted close
combined_df = combined_df.drop(columns=["Close"])

In [19]:
# rename adjusted close to close
combined_df = combined_df.rename(columns={"Adj Close": "Close"})

### Add new columns

Min-Max (0-1) Normalized closing price: ['Close Minmax']

Indexed (Starting from 1) Normalized closing price: ['Close Indexed']

In [20]:
# min-max normalization of Close price per company
combined_df['Close_minmax'] = (
    combined_df.groupby('Company')['Close']
               .transform(lambda x: (x - x.min()) / (x.max() - x.min()))
)
combined_df.head(3)


,Open,High,Low,Close,Volume,Date,Company,Close_minmax
0,58.687500,59.312500,56.000000,36.361576,53228400,2000-01-03,Microsoft,0.076244
5839,0.936384,1.004464,0.907924,0.850643,535796800,2000-01-03,Apple,0.003608
11678,4.075000,4.478125,3.952344,4.468750,322352000,2000-01-03,Amazon,0.022388


In [21]:
# indexing normalization of Close price per company
combined_df['Close_indexed'] = (
    combined_df.groupby('Company')['Close']
               .transform(lambda x: x / x.iloc[0])
)
combined_df.head(3)

,Open,High,Low,Close,Volume,Date,Company,Close_minmax,Close_indexed
0,58.687500,59.312500,56.000000,36.361576,53228400,2000-01-03,Microsoft,0.076244,1.0
5839,0.936384,1.004464,0.907924,0.850643,535796800,2000-01-03,Apple,0.003608,1.0
11678,4.075000,4.478125,3.952344,4.468750,322352000,2000-01-03,Amazon,0.022388,1.0


### Export processed data

In [22]:
combined_df.to_csv("data/processed/maang_combined.csv", index=False)